In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [3]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\HIV")

In [4]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [5]:
Pubchem_data=pd.read_csv("./Pubchem_data.csv",sep=',')
Iso_smiles=Pubchem_data['Iso_smiles']
print_RxC(Pubchem_data)
Pubchem_data.head()

11406 Moleculars × 3 Descriptors


,cid,Iso_smiles,Type
0,5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine
1,13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine
2,39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
3,56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
4,67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine


In [6]:
Smis=[]
for i in Iso_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Pubchem_data['Canonical_smiles']=Canonical_smis

In [7]:
#Load the data
Raw_data= pd.read_csv('./Original_data.csv',index_col=0)
X_scaled_data= pd.read_csv('./X_scaled_data.csv',index_col=0)
Transformer= joblib.load(filename='./Scaler_transformer.pkl')

In [8]:
Pubchem_data=Pubchem_data.drop_duplicates(subset="cid")
Pubchem_data=Pubchem_data.drop_duplicates(subset="Canonical_smiles")
Pubchem_data=Pubchem_data[~ Pubchem_data['Canonical_smiles'].isin(Raw_data['Canonical_smiles'])]
Pubchem_data.index=Pubchem_data['cid']
Pubchem_data.drop('cid',axis = 1,inplace = True)
print_RxC(Pubchem_data)
Pubchem_data.to_csv("./Pubchem_data_clearn.csv",sep=',',header=1,index=1)
Pubchem_data.head()

7268 Moleculars × 3 Descriptors


,Iso_smiles,Type,Canonical_smiles
cid,,,
5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine,Nc1ncnc2c1[nH]c[n+]2C1CCCO1
13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine,Nc1ncnc2c1ncn2C1OC(CCl)C(O)C1O
39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,CN(C)C1C(O)[C@H](n2cnc3c(N)ncnc32)O[C@@H]1CO
56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)CC1F
67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine,Nc1ncnc2c1[n+]([O-])cn2[C@@H]1O[C@H](CO)[C@@H]...


In [9]:
X_scaled_data

,IC2,IC3,IDDE,F05[N-O],IC4,Vindex,Xindex,SHED_AL,IC5,SM6_B(v),...,GATS3i,MATS7i,SpMAD_EA(bo),CATS2D_02_AP,nS,GATS8v,s2_phRelSize_2,C-033,Eta_epsi_4,s2_pathLength
cid,,,,,,,,,,,,,,,,,,,,,
395575,0.632912,0.647209,0.524016,0.000000,0.574759,0.001753,0.002566,0.587693,0.535935,0.488822,...,0.664773,0.386765,0.622626,0.000000,0.00,0.453823,0.285859,0.0,0.397836,0.416667
15604702,0.687508,0.583118,0.321200,0.049180,0.508924,0.004050,0.005750,0.280869,0.462597,0.352709,...,0.510544,0.309712,0.858462,0.111111,0.25,0.545744,0.000000,0.0,0.387818,0.000000
16045493,0.744037,0.725700,0.409965,0.032787,0.633365,0.001925,0.002976,0.644351,0.575711,0.428686,...,0.517887,0.342645,0.829876,0.111111,0.00,0.371304,0.000000,0.0,0.465825,0.000000
146894788,0.837700,0.850057,0.508259,0.049180,0.769066,0.001296,0.002032,0.943130,0.699059,0.485042,...,0.580766,0.345767,0.609660,0.000000,0.00,0.399689,0.055556,0.0,0.213139,0.500000
69480289,0.747104,0.680838,0.375343,0.032787,0.594211,0.002074,0.003161,0.584483,0.540120,0.394648,...,0.581470,0.366048,0.650762,0.000000,0.00,0.478147,0.000000,0.0,0.343705,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44582935,0.585295,0.521916,0.237022,0.000000,0.455510,0.003162,0.004546,0.280850,0.414045,0.324127,...,0.553415,0.552234,0.718804,0.000000,0.25,0.428399,0.000000,0.0,0.231863,0.250000
44583099,0.618452,0.567559,0.252696,0.000000,0.495345,0.003240,0.004634,0.283619,0.450255,0.284291,...,0.458773,0.272648,0.515121,0.000000,0.25,0.506074,0.000000,0.0,0.305645,0.250000
44583063,0.594654,0.548870,0.263487,0.000000,0.479033,0.003246,0.004640,0.203265,0.435428,0.330393,...,0.571923,0.199185,0.726820,0.000000,0.50,0.463860,0.000000,0.0,0.344872,0.250000


In [10]:
Canonical_smis=Pubchem_data['Canonical_smiles']

In [11]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_all=aDesc.get_output()

In [12]:
Data_all=pd.DataFrame(Data_all)
Desc_names_names=aDesc.get_output_descriptors()
Data_all.columns=Desc_names_names
Data_all.index=Pubchem_data.index

In [13]:
X_test=Data_all[X_scaled_data.columns]
X_test.head()

,IC2,IC3,IDDE,F05[N-O],IC4,Vindex,Xindex,SHED_AL,IC5,SM6_B(v),...,GATS3i,MATS7i,SpMAD_EA(bo),CATS2D_02_AP,nS,GATS8v,s2_phRelSize_2,C-033,Eta_epsi_4,s2_pathLength
cid,,,,,,,,,,,,,,,,,,,,,
5273,4.310443,4.458591,3.373557,0.0,4.458591,0.321832,0.467015,5.298702,4.458591,8.542473,...,1.191298,-0.326495,1.970712,3.0,0.0,0.915760,0.000000,0.0,0.602299,2.000000
13463,4.542748,4.760648,3.826875,2.0,4.825164,0.292642,0.432983,4.710651,4.825164,8.711500,...,1.182661,-0.113302,1.898385,1.0,0.0,1.206047,0.562500,0.0,0.681385,1.500000
39148,4.631305,4.733870,4.106603,3.0,4.733870,0.284324,0.422586,0.000000,4.733870,8.760417,...,1.295026,-0.287220,1.873976,1.0,0.0,1.089069,0.583333,0.0,0.619512,2.000000
56328,4.706891,4.706891,3.725481,1.0,4.706891,0.293349,0.434203,3.779763,4.706891,8.656549,...,1.113839,0.053539,1.910756,1.0,0.0,1.249478,0.166667,0.0,0.670238,1.333333
67358,4.574401,4.862576,4.121928,3.0,4.923182,0.295611,0.435668,0.000000,4.923182,8.755296,...,1.204359,-0.109247,1.900149,2.0,0.0,1.265280,0.750000,0.0,0.695238,1.500000


In [14]:
X_test_scaled=Transformer.transform(X_test)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_test_scaled.columns=X_test.columns
X_test_scaled.index=X_test.index
X_test_scaled.to_csv("./X_test_data.csv",sep=',',header=1,index=1)
print_RxC(X_test_scaled)
X_test_scaled.head()

7268 Moleculars × 1590 Descriptors


,IC2,IC3,IDDE,F05[N-O],IC4,Vindex,Xindex,SHED_AL,IC5,SM6_B(v),...,GATS3i,MATS7i,SpMAD_EA(bo),CATS2D_02_AP,nS,GATS8v,s2_phRelSize_2,C-033,Eta_epsi_4,s2_pathLength
cid,,,,,,,,,,,,,,,,,,,,,
5273,0.616671,0.521251,0.193033,0.000000,0.454929,0.003547,0.005010,0.290379,0.413517,0.236715,...,0.603316,0.126944,0.752193,0.333333,0.0,0.380561,0.000000,0.0,0.443242,0.250000
13463,0.679473,0.594902,0.270669,0.032787,0.532938,0.003225,0.004645,0.258153,0.484426,0.275319,...,0.597886,0.279268,0.679353,0.111111,0.0,0.501195,0.562500,0.0,0.656510,0.187500
39148,0.703414,0.588372,0.318576,0.049180,0.513510,0.003133,0.004534,0.000000,0.466766,0.286492,...,0.668525,0.155006,0.654770,0.111111,0.0,0.452583,0.583333,0.0,0.489660,0.250000
56328,0.723848,0.581794,0.253304,0.016393,0.507769,0.003233,0.004658,0.207138,0.461547,0.262769,...,0.554621,0.398473,0.691811,0.111111,0.0,0.519243,0.166667,0.0,0.626450,0.166667
67358,0.688030,0.619755,0.321200,0.049180,0.553797,0.003258,0.004674,0.000000,0.503386,0.285322,...,0.611527,0.282165,0.681128,0.222222,0.0,0.525810,0.750000,0.0,0.693866,0.187500
